# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [16]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [17]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head(5)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Meulaboh,ID,74.10,overcast clouds,4.1363,96.1285,Hotel Meuligou
1,2,Atar,MR,76.57,clear sky,20.5169,-13.0499,فندق سكليل
2,8,Gold Coast,AU,83.16,overcast clouds,-28.0000,153.4333,Zenith
3,9,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Pono Kai Resort
4,11,Sotouboua,TG,75.61,clear sky,8.5667,0.9833,Verger


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [18]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [25]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City_ID"] == 305]
vacation_end = vacation_df.loc[vacation_df["City_ID"] == 305]
vacation_stop1 = vacation_df.loc[vacation_df["City_ID"] == 346]
vacation_stop2 = vacation_df.loc[vacation_df["City_ID"] == 578]
vacation_stop3 = vacation_df.loc[vacation_df["City_ID"] == 600]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [30]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2,
                                         vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
73,305,Acapulco,MX,78.15,scattered clouds,16.8634,-99.8901,Hotel del Valle
82,346,Isla Mujeres,MX,75.25,clear sky,21.2311,-86.7310,Villa La Bella
127,578,Carmen,MX,78.84,few clouds,18.6333,-91.8333,Holiday Inn Express
133,600,Lazaro Cardenas,MX,77.79,clear sky,17.9583,-102.2000,Hotel Sol del Pacífico
73,305,Acapulco,MX,78.15,scattered clouds,16.8634,-99.8901,Hotel del Valle


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [58]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lng", "Lat"]].copy()

# Display sample data
waypoints_df

,Lng,Lat
73,-99.8901,16.8634
82,-86.7310,21.2311
127,-91.8333,18.6333
133,-102.2000,17.9583
73,-99.8901,16.8634


## Use GeoViews to create map that shows the four cities in the itinerary

In [59]:
# Configure the map plot by using the itineraty_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "purple")

In [60]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [61]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [62]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '48bfb7c90c424406b8af14821aab4a04',
 'waypoints': '16.8634,-99.8901|21.2311,-86.731|18.6333,-91.8333|17.9583,-102.2|16.8634,-99.8901'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [100]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-99.8901, 16.8634], 'original_index': 0},
     {'location': [-86.731, 21.2311], 'original_index': 1},
     {'location': [-91.8333, 18.6333], 'original_index': 2},
     {'location': [-102.2, 17.9583], 'original_index': 3},
     {'location': [-99.8901, 16.8634], 'original_index': 4}],
    'units': 'metric',
    'distance': 4491312,
    'distance_units': 'meters',
    'time': 167823.755,
    'legs': [{'distance': 1915353,
      'time': 71542.234,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 30,
        'time': 3.676,
        'instruction': {'text': 'Drive south on Calle Vicente Guerrero.'}},
       {'from_index': 1,
        'to_index': 3,
        'distance': 44,
        'time': 11.016,
        'instruction': {'text': 'Turn left onto Avenida Sonora.'}},
       {'from_index': 3,
        'to_index': 5,
        'distance': 208,
        'time': 27.305,
        'i

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [127]:
import json
legs = route_resources["legs"][0]["distance"]

NameError: name 'route_resources' is not defined

In [12]:
# Fetch the route's legs coordinates from the JSON reponse
legs = # YOUR CODE HERE

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [13]:
# Create and empty list to store the longitude of each step
# YOUR CODE HERE

# Create and empty list to store the latitude of step
# YOUR CODE HERE

# Loop through the legs coordinates to fetch the latitude and longitude for each step
# YOUR CODE HERE

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [14]:
# Create an empty DataFrame to store the steps' coordinates
route_df = # YOUR CODE HERE

# Add the steps' longitude and latitude from each step as columns to the DataFrame
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,longitude,latitude
0,12.479944,41.889982
1,12.479927,41.890012
2,12.479760,41.890249
3,12.479580,41.890493
4,12.479418,41.890662


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [15]:
# Configure the route path by using the GeoViews' Path function
route_path = # YOUR CODE HERE

In [16]:
# Display a composed plot by using the route_map and route_path objects
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]